# Model with Metrics

Example testing a model with custom metrics.

Metrics can be 

  * A ```COUNTER``` : the returned value will increment the current value
  * A ```GAUGE``` : the returned value will overwrite the current value
  * A ```TIMER``` : a number of millisecs. Prometheus SUM and COUNT metrics will be created.
  
You need to provide a list of dictionaries each with the following:

  * a ```type``` : COUNTER, GAUGE, or TIMER
  * a ```key``` : a user defined key
  * a ```value``` : a float value
  
See example code below:
 

In [1]:
!pygmentize ModelWithMetrics.py

class ModelWithMetrics(object):

    def __init__(self):
        print("Initialising")

    def predict(self,X,features_names):
        print("Predict called")
        return X

    def metrics():
        return [
            {"type":"COUNTER","key":"mycounter","value":1}, # a counter which will increase by the given value
            {"type":"GAUGE","key":"mygauge","value":100}, # a gauge which will be set to given value
            {"type":"TIMER","key":"mytimer","value":20.2}, # a timer which will add sum and count metrics - assumed millisecs
            ]
    


# REST

In [11]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-rest:0.1

---> Installing application source...
Build completed successfully


In [5]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-rest:0.1

d23013897d05f9b5028ddc8825eb8d2d90073e6ecbfda50d8d5e5e556f5453c9


In [9]:
!cd ../../../wrappers/testing && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
rm -f fbs/*.py
rm -rf fbs/__pycache__
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [12]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[7.508, 4.0, 6.443, 2.41]]}}
RECEIVED RESPONSE:
{'data': {'names': ['t:0', 't:1', 't:2', 't:3'], 'ndarray': [[7.508, 4.0, 6.443, 2.41]]}, 'meta': {'metrics': [{'key': 'mycounter', 'type': 'COUNTER', 'value': 1}, {'key': 'mygauge', 'type': 'GAUGE', 'value': 100}, {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]}}

Time 0.0059931278228759766


In [13]:
!docker rm model-with-metrics --force

model-with-metrics


# gRPC

In [22]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-grpc:0.1

---> Installing application source...
Build completed successfully


In [23]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-grpc:0.1

4e68f0b1c988632b8811168983e21f724015c622d047d02fdeaeb8a06ed3e74d


In [16]:
!cd ../../../wrappers/testing && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
rm -f fbs/*.py
rm -rf fbs/__pycache__
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [24]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  ndarray {
    values {
      list_value {
        values {
          number_value: 7.428
        }
        values {
          number_value: 3.564
        }
        values {
          number_value: 5.525
        }
        values {
          number_value: 1.467
        }
      }
    }
  }
}

RECEIVED RESPONSE:
meta {
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
    value: 20.200000762939453
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  ndarray {
    values {
      list_value {
        values {
          number_value: 7.428
        }
        values {
          number_value: 3.564
        }
        values {
          number_value: 5.525
        }
        values {
          number

In [25]:
!docker rm model-with-metrics --force

model-with-metrics


# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need Minikube version 0.25.2**

In [26]:
!minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

There is a newer version of minikube available (v0.30.0).  Download it here:
https://github.com/kubernetes/minikube/releases/tag/v0.30.0

To disable this notification, run the following:
minikube config set WantUpdateNotification false
Starting local Kubernetes v1.9.4 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [27]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [28]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [29]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [30]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Nov  3 08:25:19 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Sat Nov  3 08:25:20 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME            AGE
seldon-default  0s

==> v1beta1/Ro

In [36]:
!helm install seldon-core-analytics --name seldon-core-analytics --set grafana_prom_admin_password=password --set persistence.enabled=false --repo https://storage.googleapis.com/seldon-charts 

NAME:   seldon-core-analytics
LAST DEPLOYED: Sat Nov  3 08:31:02 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>         1s

==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     1s

==> v1/ServiceAccount
NAME        SECRETS  AGE
prometheus  1        1s

==> v1beta1/ClusterRole
NAME        AGE
prometheus  1s

==> v1beta1/ClusterRoleBinding
NAME        AGE
prometheus  1s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          1s
grafana-prom-deployment  1        1        1           0          1s
prometheus-deployment    1        1        1           0          1s

==> v1/Service
NAME                      TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)   

# REST

In [39]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-rest:0.1 --loglevel 5

I1103 08:44:32.398667   32003 build.go:50] Running S2I version "v1.1.12"
I1103 08:44:32.398790   32003 util.go:58] Getting docker credentials for seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1103 08:44:32.398809   32003 util.go:74] Using  credentials for pulling seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1103 08:44:32.429416   32003 docker.go:487] Using locally available image "seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT"
I1103 08:44:32.430628   32003 build.go:163] 
Builder Image:			seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
Source:				.
Output Image Tag:		model-with-metrics-rest:0.1
Environment:			PERSISTENCE=0,MODEL_NAME=ModelWithMetrics,API_TYPE=REST,SERVICE_TYPE=MODEL
Environment File:		environment_rest
Labels:				
Incremental Build:		disabled
Remove Old Build:		disabled
Builder Pull Policy:		if-not-present
Previous Image Pull Policy:	if-not-present
Quiet:				disabled
Layered Build:			disabled
Docker Endpoint:		tcp://192.168.39.12:2376
Docker Pull Config:		/home/clive/.d

I1103 08:44:32.760107   32003 docker.go:1057] Invoking PostExecute function
I1103 08:44:32.760121   32003 postexecutorstep.go:68] Skipping step: store previous image
I1103 08:44:32.760124   32003 postexecutorstep.go:117] Executing step: commit image
I1103 08:44:32.761864   32003 postexecutorstep.go:522] Checking for new Labels to apply... 
I1103 08:44:32.761878   32003 postexecutorstep.go:530] Creating the download path '/tmp/s2i235039620/metadata'
I1103 08:44:32.761941   32003 postexecutorstep.go:464] Downloading file "/tmp/.s2i/image_metadata.json"
I1103 08:44:32.790762   32003 postexecutorstep.go:538] unable to download and extract 'image_metadata.json' ... continuing
I1103 08:44:32.793620   32003 docker.go:1091] Committing container with dockerOpts: {Reference:model-with-metrics-rest:0.1 Comment: Author: Changes:[] Pause:false Config:0xc4203baf00}, config: {Hostname: Domainname: User: AttachStdin:false AttachStdout:false AttachStderr:false ExposedPorts:map[] Tty:false OpenStdin:fal

In [40]:
!kubectl create -f deployment-rest.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [33]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[name:mymodel-mymodel-svc-orch replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:mymodel-mymodel-complex-model-0]] state:Available]

In [34]:
!cd ../../../util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [45]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[5.314392890434304, 2.0870503600642127, 7.4093196094849265, 2.808545702671992]]}}
Getting token from http://192.168.39.12:30088/oauth/token
{"access_token":"07b7e438-8d7e-48b6-82d2-9d310f515757","token_type":"bearer","expires_in":42150,"scope":"read write"}
RECEIVED RESPONSE:
{'meta': {'puid': 's5evocddmdohugpqqv5iddc7po', 'tags': {}, 'routing': {}, 'requestPath': {'complex-model': 'model-with-metrics-rest:0.1'}, 'metrics': [{'key': 'mycounter', 'type': 'COUNTER', 'value': 1.0}, {'key': 'mygauge', 'type': 'GAUGE', 'value': 100.0}, {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]}, 'data': {'names': ['t:0', 't:1', 't:2', 't:3'], 'ndarray': [[5.314392890434304, 2.0870503600642127, 7.4093196094849265, 2.808545702671992]]}}



In [46]:
!kubectl delete -f deployment-rest.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


# gRPC

In [47]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-grpc:0.1

---> Installing application source...
Build completed successfully


In [48]:
!kubectl create -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [49]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[replicas:1 replicasAvailable:1 name:mymodel-mymodel-svc-orch] map[name:mymodel-mymodel-complex-model-0 replicas:1 replicasAvailable:1]] state:Available]

In [50]:
!cd ../../../util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [52]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  ndarray {
    values {
      list_value {
        values {
          number_value: 7.294562576598512
        }
        values {
          number_value: 3.6624031290151073
        }
        values {
          number_value: 2.755651138726249
        }
        values {
          number_value: 1.2337746487942007
        }
      }
    }
  }
}

Getting token from http://192.168.39.12:30088/oauth/token
{"access_token":"07b7e438-8d7e-48b6-82d2-9d310f515757","token_type":"bearer","expires_in":40402,"scope":"read write"}
RECEIVED RESPONSE:
meta {
  puid: "if7btsi0i34p44tjfaj6adkst"
  requestPath {
    key: "complex-model"
    value: "model-with-metrics-grpc:0.1"
  }
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
   

## Test feedback

In [40]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --endpoint send-feedback --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

----------------------------------------
SENDING NEW REQUEST:
request {
  data {
    names: "sepal_length"
    names: "sepal_width"
    names: "petal_length"
    names: "petal_width"
    ndarray {
      values {
        list_value {
          values {
            number_value: 4.892674496650631
          }
          values {
            number_value: 4.742377470286767
          }
          values {
            number_value: 2.851798122512135
          }
          values {
            number_value: 0.004773596258239476
          }
        }
      }
    }
  }
}
response {
  data {
    names: "class1"
    names: "class2"
    names: "class3"
    ndarray {
      values {
        list_value {
          values {
            number_value: 0.8873412759534708
          }
          values {
            number_value: 0.6945005393322701
          }
          values {
            number_value: 0.18687912951259333
          }
        }
      }
    }
  }
}
reward: 1.0

Getting token from http://192.16

In [41]:
!kubectl delete -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


In [42]:
!minikube delete

Deleting local Kubernetes cluster...
Machine deleted.
